In [118]:
import pandas as pd
import sqlite3
from Levenshtein import distance

connect = sqlite3.connect("raw-database.db")
cursor = connect.cursor()

req = cursor.execute("SELECT id, area, name from compet_SK WHERE gender = 'male' AND age_group = 'adult'")
res = req.fetchall()
desc = req.description
df_compet_SK = pd.DataFrame(res)
df_compet_SK.columns = [i[0] for i in desc]

df_compet_SK = df_compet_SK[[("Play" not in i or "play" not in i) and ("Off" not in i or "off" not in i) for i in df_compet_SK.name]]
df_compet_SK.set_index("id", inplace = True)
df_compet_SK = pd.DataFrame(df_compet_SK.area + " - " + df_compet_SK.name, columns = ["compet_SK"]).reset_index()

req = cursor.execute("SELECT DISTINCT competition_id, country_name, competition_name from compet_SB WHERE competition_youth = False AND competition_youth = False")
res = req.fetchall()
desc = req.description
df_compet_SB = pd.DataFrame(res)
df_compet_SB.columns = [i[0] for i in desc]

connect.close()

df_compet_SB = df_compet_SB[["Play-offs" not in i for i in df_compet_SB.competition_name]]
df_compet_SB.set_index("competition_id", inplace = True)
df_compet_SB = pd.DataFrame(df_compet_SB.country_name + " - " + df_compet_SB.competition_name, columns = ["compet_SB"]).reset_index()

In [199]:
liste_compet_SK = df_compet_SK.compet_SK
df_distance = pd.DataFrame(index = liste_compet_SK)
for compet_SB in df_compet_SB.compet_SB :
    df_distance[compet_SB] = [distance(compet_SB, i, weights = (1, 1, 5)) for i in liste_compet_SK]
dico = {}
min_distance = df_distance.min().sort_values()
min_distance = min_distance[min_distance < 20].index
df_distance = df_distance[min_distance]
i = 0
while min(df_distance.shape) != 0 :
    compet_SB = min_distance[i]
    compet_SK = df_distance[compet_SB].idxmin()
    dico[compet_SB] = compet_SK
    df_distance.drop(compet_SK, axis = 0, inplace = True)
    df_distance.drop(compet_SB, axis = 1, inplace = True)
    i += 1

df_final = pd.DataFrame(dico.items(), columns = ["compet_SB", "compet_SK"])
df_final = pd.merge(df_final, df_compet_SB, on = "compet_SB")
df_final = pd.merge(df_final, df_compet_SK, on = "compet_SK")
df_final = pd.concat([df_final, df_compet_SK[~df_compet_SK.compet_SK.isin(df_final.compet_SK)]])
df_final = pd.concat([df_final, df_compet_SB[~df_compet_SB.compet_SB.isin(df_final.compet_SB)]])

In [200]:
df_final

,compet_SB,compet_SK,competition_id,id
0,Norway - Eliteserien,NOR - Eliteserien,88.0,124.0
1,France - Ligue 1,FRA - Ligue 1,7.0,3.0
2,France - Ligue 2,FRA - Ligue 2,8.0,12.0
3,Austria - Bundesliga,AUT - Bundesliga,47.0,88.0
4,Portugal - Segunda Liga,POR - Segunda Liga,357.0,102.0
5,Europe - UEFA Europa League,UEFA - Europa League,35.0,9.0
6,Europe - UEFA Europa Conference League,UEFA - Europa Conference League,353.0,131.0
7,Netherlands - Eredivisie,NED - Eredivisie,6.0,17.0
8,Belgium - Jupiler Pro League,BEL - Pro League,46.0,16.0
9,Portugal - Liga NOS,POR - Primeira Liga,13.0,25.0


In [179]:
df_final

,compet_SB,compet_SK,competition_id,id
0,Norway - Eliteserien,NOR - Eliteserien,88.0,124.0
1,France - Ligue 1,FRA - Ligue 1,7.0,3.0
2,France - Ligue 2,FRA - Ligue 2,8.0,12.0
3,Austria - Bundesliga,AUT - Bundesliga,47.0,88.0
4,Portugal - Segunda Liga,POR - Segunda Liga,357.0,102.0
5,Europe - UEFA Europa League,UEFA - Europa League,35.0,9.0
6,Europe - UEFA Europa Conference League,UEFA - Europa Conference League,353.0,131.0
7,Netherlands - Eredivisie,NED - Eredivisie,6.0,17.0
8,Belgium - Jupiler Pro League,BEL - Pro League,46.0,16.0
9,Portugal - Liga NOS,POR - Primeira Liga,13.0,25.0


In [116]:
df_final = pd.DataFrame(dico.items(), columns = ["compet_SB", "compet_SK"])
df_final = pd.merge(df_final, df_compet_SB, on = "compet_SB")
df_final = pd.merge(df_final, df_compet_SK, on = "compet_SK")
df_final = pd.concat([df_final, df_compet_SK[~df_compet_SK.compet_SK.isin(df_final.compet_SK)]])
df_final.sort_values(by = "competition_id").dropna(subset = "compet_SB")

,compet_SB,compet_SK,competition_id,id
9,Netherlands - Eredivisie,NED - Eredivisie,6.0,17
2,France - Ligue 1,FRA - Ligue 1,7.0,3
3,France - Ligue 2,FRA - Ligue 2,8.0,12
10,Portugal - Liga NOS,POR - Liga 3,13.0,162
8,Europe - UEFA Europa League,UEFA - Europa League,35.0,9
12,Belgium - Jupiler Pro League,BEL - Pro League,46.0,16
5,Austria - Bundesliga,AUT - Bundesliga,47.0,88
11,Europe - UEFA Euro,ENG - FA Cup,55.0,19
1,France - Coupe de France,FRA - Coupe de France,86.0,7
0,Norway - Eliteserien,NOR - Eliteserien,88.0,124


In [104]:
df_compet_SK.sort_values(by = "id").sort_values(by = "id")

,id,compet_SK
81,0,FIFA - World Cup
73,1,ENG - Premier League
179,2,UEFA - European Championship
85,3,FRA - Ligue 1
75,4,ESP - LaLiga
...,...,...
18,408,BIH - Premier League
129,409,MAR - Botola Pro
9,412,ARG - Primera Nacional
120,439,ITA - Serie D - Girone D
